In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sbn
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error

%matplotlib inline

In [2]:
df = pd.read_csv('Data for Cleaning & Modeling.csv')

C:\Users\jerem\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400000 entries, 0 to 399999
Data columns (total 32 columns):
X1     338990 non-null object
X2     399999 non-null float64
X3     399999 non-null float64
X4     399999 non-null object
X5     399999 non-null object
X6     399999 non-null object
X7     399999 non-null object
X8     338730 non-null object
X9     338730 non-null object
X10    376014 non-null object
X11    382462 non-null object
X12    338639 non-null object
X13    338972 non-null float64
X14    399999 non-null object
X15    399999 non-null object
X16    123560 non-null object
X17    399999 non-null object
X18    399981 non-null object
X19    399999 non-null object
X20    399999 non-null object
X21    399999 non-null float64
X22    399999 non-null float64
X23    399999 non-null object
X24    399999 non-null float64
X25    181198 non-null float64
X26    51155 non-null float64
X27    399999 non-null float64
X28    399999 non-null float64
X29    399999 non-null float64
X30    39

Rename column names for greater readability

In [4]:
column_names = ['interest_rate','loan_id','borrower_id','amount_requested','amount_funded',
 'investor_funded_portion','number_of_payments','loan_grade','loan_subgrade',
 'employer_or_jobtitle','years_employed','home_ownership_status','annual_income',
 'income_verification_status','issue_date','loan_reason','loan_category','loan_title',
 'zip_code','state','debt_to_income_ratio','number_of_deliquencies','earliest_credit_line_date',
 'number_of_inquiries','months_since_last_delinquency','months_since_last_public_record',
 'number_of_open_credit_lines','number_of_derogatory_public_records','total_revolving_credit',
 'line_utilization_rate','number_of_credit_lines','listing_status']

In [5]:
df.columns = column_names

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400000 entries, 0 to 399999
Data columns (total 32 columns):
interest_rate                          338990 non-null object
loan_id                                399999 non-null float64
borrower_id                            399999 non-null float64
amount_requested                       399999 non-null object
amount_funded                          399999 non-null object
investor_funded_portion                399999 non-null object
number_of_payments                     399999 non-null object
loan_grade                             338730 non-null object
loan_subgrade                          338730 non-null object
employer_or_jobtitle                   376014 non-null object
years_employed                         382462 non-null object
home_ownership_status                  338639 non-null object
annual_income                          338972 non-null float64
income_verification_status             399999 non-null object
issue_date        

Drop rows with missing interest rates, line utilization rate, and loan_id

In [7]:
loan_df = df[pd.notnull(df.loan_id) & pd.notnull(df.interest_rate) & pd.notnull(df.line_utilization_rate)].copy()

Convert amount requested, line utilization rate, and interest rate fields into numeric types

In [8]:
loan_df.amount_requested = loan_df.amount_requested.apply(lambda x: float(x.replace('$','').replace(',','')))
loan_df.line_utilization_rate = loan_df.line_utilization_rate.apply(lambda x: float(x.replace('%','')) if pd.notnull(x) else x)
loan_df.interest_rate = loan_df.interest_rate.apply(lambda x: float(x.replace('%','')))

Extract date fields

In [9]:
loan_df['issue_year'] = loan_df.issue_date.apply(lambda x: int(x.split('-')[1]))

month_map = {'Jan':1,'Feb':2,'Mar':3,'Apr':4,'May':5,'Jun':6,'Jul':7,'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12}
loan_df['issue_month'] = loan_df.issue_date.apply(lambda x: month_map[x.split('-')[0]])

Fill missing annual income values

In [10]:
annual_income_lkup1 = loan_df.groupby(['state','zip_code','employer_or_jobtitle'])['annual_income'].median().dropna().to_dict()
annual_income_lkup2 = loan_df.groupby(['state','employer_or_jobtitle'])['annual_income'].median().dropna().to_dict()
annual_income_lkup3 = loan_df.groupby(['state','zip_code'])['annual_income'].median().dropna().to_dict()
annual_income_lkup4 = loan_df.groupby(['state'])['annual_income'].median().to_dict()

def fill_annual_income(state, zip_code, employment):
    if (state, zip_code, employment) in annual_income_lkup1:
        return annual_income_lkup1[(state, zip_code, employment)]
    elif (state, employment) in annual_income_lkup2:
        return annual_income_lkup2[(state, employment)]
    elif (state, zip_code) in annual_income_lkup3:
        return annual_income_lkup3[(state, zip_code)]
    else:
        return annual_income_lkup4[state]

loan_df.annual_income = loan_df.apply(lambda row: fill_annual_income(row['state'],row['zip_code'],row['employer_or_jobtitle']) if pd.isnull(row['annual_income']) else row['annual_income'], axis=1)

Fill missing deliquency values

In [11]:
loan_df['had_deliquency'] = loan_df.months_since_last_delinquency.apply(lambda x: 1 if pd.notnull(x) else 0)
loan_df.months_since_last_delinquency = loan_df.months_since_last_delinquency.fillna(0)

Create new feature for length of credit history

In [12]:
loan_df['credit_age'] = loan_df.earliest_credit_line_date.apply(lambda x: 2016 - datetime.strptime(x,'%b-%y').year)

Convert listing status and number of payments into boolean field

In [13]:
loan_df.listing_status = loan_df.listing_status.apply(lambda x: 1 if x == 'f' else 0)
loan_df.number_of_payments = loan_df.number_of_payments.apply(lambda x: 1 if x == ' 36 months' else 0)

Fill loan grade using KNN

In [14]:
subgrade_map = {g:n+1 for n,g in enumerate(sorted(loan_df[loan_df.loan_subgrade.notnull()].loan_subgrade.unique()))}
loan_df.loan_subgrade = loan_df.loan_subgrade.apply(lambda x: subgrade_map[x] if x in subgrade_map else x)

knn_clf = KNeighborsClassifier(n_neighbors=5)

knn_features = ['interest_rate']

knn_df = loan_df[loan_df.loan_subgrade.notnull()][knn_features]

Y_knn = loan_df[loan_df.loan_subgrade.notnull()]['loan_subgrade']

knn_clf.fit(knn_df,Y_knn)

null_grades = loan_df[loan_df.loan_subgrade.isnull()][['interest_rate','loan_subgrade']].copy()
null_grades['loan_subgrade'] = knn_clf.predict(null_grades.interest_rate.values.reshape(-1,1))
loan_df.update(null_grades)

In [15]:
features = ['amount_requested','number_of_payments','annual_income','loan_subgrade',
            'issue_year','issue_month','credit_age', 'debt_to_income_ratio','line_utilization_rate','had_deliquency',
            'months_since_last_delinquency','number_of_open_credit_lines','number_of_inquiries', 'total_revolving_credit',
            'number_of_derogatory_public_records','listing_status']

In [16]:
features_df = loan_df[features]
Y = loan_df['interest_rate']

X_train, X_test, y_train, y_test = train_test_split(features_df, Y, test_size=0.3, random_state=50)

In [17]:
linear_reg = LinearRegression()
linear_reg.fit(X_train,y_train)
pred = linear_reg.predict(X_test)

print('Linear Regression RMSE {0}'.format(mean_squared_error(y_test,pred)**0.5))

Linear Regression RMSE 0.9087777733683705


In [18]:
rf_reg = RandomForestRegressor(n_estimators=100)
rf_reg.fit(X_train, y_train)
pred = rf_reg.predict(X_test)

print('Random Forest RMSE {0}'.format(mean_squared_error(y_test,pred)**0.5))

Random Forest RMSE 0.24078972444603933
